<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=400/>
        <br/>
    <b style="font-size: 30px">Python byte code</b>
    <br/>
    <br/>
    <br/>
<b style="font-size: 20px">Алексей Стыценко</b>
</center>

Возьмём простую программу

In [15]:
program = """\
x = 1
y = 2
print(3*(x+y))
"""

In [16]:
exec(program)

9


Как интерпретатор её выполняет?

Шаг 1: Токенизация

In [17]:
import io
import tokenize

list(tokenize.generate_tokens(io.StringIO(program).readline))

[TokenInfo(type=1 (NAME), string='x', start=(1, 0), end=(1, 1), line='x = 1\n'),
 TokenInfo(type=54 (OP), string='=', start=(1, 2), end=(1, 3), line='x = 1\n'),
 TokenInfo(type=2 (NUMBER), string='1', start=(1, 4), end=(1, 5), line='x = 1\n'),
 TokenInfo(type=4 (NEWLINE), string='\n', start=(1, 5), end=(1, 6), line='x = 1\n'),
 TokenInfo(type=1 (NAME), string='y', start=(2, 0), end=(2, 1), line='y = 2\n'),
 TokenInfo(type=54 (OP), string='=', start=(2, 2), end=(2, 3), line='y = 2\n'),
 TokenInfo(type=2 (NUMBER), string='2', start=(2, 4), end=(2, 5), line='y = 2\n'),
 TokenInfo(type=4 (NEWLINE), string='\n', start=(2, 5), end=(2, 6), line='y = 2\n'),
 TokenInfo(type=1 (NAME), string='print', start=(3, 0), end=(3, 5), line='print(3*(x+y))\n'),
 TokenInfo(type=54 (OP), string='(', start=(3, 5), end=(3, 6), line='print(3*(x+y))\n'),
 TokenInfo(type=2 (NUMBER), string='3', start=(3, 6), end=(3, 7), line='print(3*(x+y))\n'),
 TokenInfo(type=54 (OP), string='*', start=(3, 7), end=(3, 8), line

Шаг 2: Построение дерева разбора

https://docs.python.org/3/reference/grammar.html

Шаг 2: Построение дерева разбора

In [18]:
import symbol
import token
import parser

# хелпер для вывода дерева разбора в читаемом виде
# https://realpython.com/cpython-source-code-guide/#lexing-and-parsing
def lex(expression):
    symbols = {v: k for k, v in symbol.__dict__.items() if isinstance(v, int)}
    tokens = {v: k for k, v in token.__dict__.items() if isinstance(v, int)}
    lexicon = {**symbols, **tokens}
    st = parser.expr(expression)
    st_list = parser.st2list(st)

    def replace(l: list):
        r = []
        for i in l:
            if isinstance(i, list):
                r.append(replace(i))
            else:
                if i in lexicon:
                    r.append(lexicon[i])
                else:
                    r.append(i)
        return r

    return replace(st_list)

Шаг 2: Построение дерева разбора

In [19]:
lex('3*(x+y)')

['eval_input',
 ['testlist',
  ['test',
   ['or_test',
    ['and_test',
     ['not_test',
      ['comparison',
       ['expr',
        ['xor_expr',
         ['and_expr',
          ['shift_expr',
           ['arith_expr',
            ['term',
             ['factor', ['power', ['atom_expr', ['atom', ['NUMBER', '3']]]]],
             ['STAR', '*'],
             ['factor',
              ['power',
               ['atom_expr',
                ['atom',
                 ['LPAR', '('],
                 ['testlist_comp',
                  ['namedexpr_test',
                   ['test',
                    ['or_test',
                     ['and_test',
                      ['not_test',
                       ['comparison',
                        ['expr',
                         ['xor_expr',
                          ['and_expr',
                           ['shift_expr',
                            ['arith_expr',
                             ['term',
                              ['factor',
     

Шаг 3: Абстрактное синтаксическое дерево (AST)

In [20]:
import ast
print(ast.dump(ast.parse(program), indent=4))

Module(
    body=[
        Assign(
            targets=[
                Name(id='x', ctx=Store())],
            value=Constant(value=1)),
        Assign(
            targets=[
                Name(id='y', ctx=Store())],
            value=Constant(value=2)),
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    BinOp(
                        left=Constant(value=3),
                        op=Mult(),
                        right=BinOp(
                            left=Name(id='x', ctx=Load()),
                            op=Add(),
                            right=Name(id='y', ctx=Load())))],
                keywords=[]))],
    type_ignores=[])


Шаг 4: Компиляция в байткод

In [21]:
code = compile(program, '<string>', 'exec')

In [22]:
code.co_code

b'd\x00Z\x00d\x01Z\x01e\x02d\x02e\x00e\x01\x17\x00\x14\x00\x83\x01\x01\x00d\x03S\x00'

In [23]:
len(code.co_code)

28

In [24]:
print(list(code.co_code))

[100, 0, 90, 0, 100, 1, 90, 1, 101, 2, 100, 2, 101, 0, 101, 1, 23, 0, 20, 0, 131, 1, 1, 0, 100, 3, 83, 0]


In [25]:
import dis
dis.opname[100], dis.opname[90], dis.opname[101], dis.opname[131]

('LOAD_CONST', 'STORE_NAME', 'LOAD_NAME', 'CALL_FUNCTION')

Байткод

In [26]:
dis.dis(program)

  1           0 LOAD_CONST               0 (1)
              2 STORE_NAME               0 (x)

  2           4 LOAD_CONST               1 (2)
              6 STORE_NAME               1 (y)

  3           8 LOAD_NAME                2 (print)
             10 LOAD_CONST               2 (3)
             12 LOAD_NAME                0 (x)
             14 LOAD_NAME                1 (y)
             16 BINARY_ADD
             18 BINARY_MULTIPLY
             20 CALL_FUNCTION            1
             22 POP_TOP
             24 LOAD_CONST               3 (None)
             26 RETURN_VALUE


Как читать вывод dis: https://stackoverflow.com/a/47529318

Что делают инструкции: https://docs.python.org/3.9/library/dis.html#python-bytecode-instructions

Байткод

In [33]:
program = '''\
def foo(x, y):
    return 3*(x + y)
foo(1, 2)
'''
dis.dis(program)

  1           0 LOAD_CONST               0 (<code object foo at 0x10e28ab30, file "<dis>", line 1>)
              2 LOAD_CONST               1 ('foo')
              4 MAKE_FUNCTION            0
              6 STORE_NAME               0 (foo)

  3           8 LOAD_NAME                0 (foo)
             10 LOAD_CONST               2 (1)
             12 LOAD_CONST               3 (2)
             14 CALL_FUNCTION            2
             16 POP_TOP
             18 LOAD_CONST               4 (None)
             20 RETURN_VALUE

Disassembly of <code object foo at 0x10e28ab30, file "<dis>", line 1>:
  2           0 LOAD_CONST               1 (3)
              2 LOAD_FAST                0 (x)
              4 LOAD_FAST                1 (y)
              6 BINARY_ADD
              8 BINARY_MULTIPLY
             10 RETURN_VALUE


Байткод

In [28]:
list(dis.get_instructions(program))

[Instruction(opname='LOAD_CONST', opcode=100, arg=0, argval=1, argrepr='1', offset=0, starts_line=1, is_jump_target=False),
 Instruction(opname='STORE_NAME', opcode=90, arg=0, argval='x', argrepr='x', offset=2, starts_line=None, is_jump_target=False),
 Instruction(opname='LOAD_CONST', opcode=100, arg=1, argval=2, argrepr='2', offset=4, starts_line=2, is_jump_target=False),
 Instruction(opname='STORE_NAME', opcode=90, arg=1, argval='y', argrepr='y', offset=6, starts_line=None, is_jump_target=False),
 Instruction(opname='LOAD_NAME', opcode=101, arg=2, argval='print', argrepr='print', offset=8, starts_line=3, is_jump_target=False),
 Instruction(opname='LOAD_CONST', opcode=100, arg=2, argval=3, argrepr='3', offset=10, starts_line=None, is_jump_target=False),
 Instruction(opname='LOAD_NAME', opcode=101, arg=0, argval='x', argrepr='x', offset=12, starts_line=None, is_jump_target=False),
 Instruction(opname='LOAD_NAME', opcode=101, arg=1, argval='y', argrepr='y', offset=14, starts_line=None, 

Ссылки

- https://leanpub.com/insidethepythonvirtualmachine/read
- https://realpython.com/cpython-source-code-guide/